In [10]:
import cv2
import os
import re
import math
import json
import random
import numpy as np
import pandas as pd
import holoviews as hv
import EzTrackFunctions as ez
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import FreezeAnalysis_Functions as fz

# Defining Functions & Variables

In [6]:
def find_file_with_strings(root_path, strings_list, dont_include_suffix):
    strings_list = [s.lower() for s in strings_list]
    print(strings_list)
    for root, dirs, files in os.walk(root_path):
        for file in files:
            # ignore files that start with a dot (annoying lock files)
            if file.startswith("."):
                continue
            else:
                file_path = os.path.join(root, file).lower()
                if all(string.lower() in file_path for string in strings_list) and dont_include_suffix.lower() not in file_path:
                    return os.path.join(root, file)
    return None

def find_file_with_strings(root_path, strings_list):
    strings_list = [s.lower() for s in strings_list]
    print(strings_list)
    for root, dirs, files in os.walk(root_path):
        for file in files:
            # ignore files that start with a dot (annoying lock files)
            if file.startswith("."):
                continue
            else:
                file_path = os.path.join(root, file).lower()
                if all(string.lower() in file_path for string in strings_list):
                    return os.path.join(root, file)
    return None

def time_to_frames(time, fps):
    time_str = str(time)
    if ":" in time_str:
        # Convert time in minutes to time in seconds
        minutes, seconds = map(int, time_str.split(":"))
        time_in_seconds = 60 * minutes + seconds
    else:
        time_in_seconds = int(time)

    # Calculate number of frames based on time and fps
    frames = int(time_in_seconds * fps)

    return frames

In [3]:
# this will determine some parameters going into the automated analysis
person = "Olena"
# Adjust these parameters based on trial and error
FreezeThresh = 180 
MinDuration = 40
# this should be the suffix of your info sheet, if not, change it
info_sheet_suffix  = "_FC_startime_info.csv"
# this should be the suffix of your eztrack output, if not, change it
eztrack_output_processed_suffix = "FreezingOutput_processed.csv"
# your root path
PERSON_ROOT = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group"
# your calibration root path, if the same as above, c&p the above
CALIBRATION_ROOT = "/media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/calibration"
# how much time after the event occurred do you want to get? in seconds
half_time_window = 2.5
# most likely you don't need to change this
calibrate_video_what_frame_to_start = 0
number_of_frames_to_calibrate = 600
h,w = 300,1000
dsmpl = 1
vid_d_start = 0
# change if you want to track another event, make sure name matchs column name
event_tracked = 'CS ON'

In [4]:
# defining columns we will be looking at for each person
experiment_cols = {
        "Olena": ["conditioning", "extinction_1", "extinction_2", "retrieval", "late_retrieval", "renewal"],
        "Ozge": ["conditioning", "extinction_1"],
        "Patrick": ["conditioning", "extinction_1", "retrieval"],
    }

general_cols = {
    "Olena":["animal_id", "camera_type"],
    "Ozge": ["animal_id"],
    "Patrick": ["animal_id"],
}

# we also focus on : chamber and experimental_group columns but we don't define that here

In [5]:
start_info_filepath = os.path.join(PERSON_ROOT, f"{person.lower()}{info_sheet_suffix}")
print(f"start_info_filepath: {start_info_filepath}")

df_start_info = pd.read_csv(start_info_filepath)
df_start_info.head()

start_info_filepath: /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/olena_FC_startime_info.csv


,animal_name,animal_id,chamber,experimental_group,camera_id,camera_type,conditioning,extinction_1,extinction_2,retrieval,late_retrieval,renewal
0,C10208,hSyn-AS-Gi-1,A,AS-Gi,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000
1,C10208,hSyn-AS-Gi-1,A,AS-Gi,1,usb,205.9097,388.3886,356.8101,208.6715,203.0164,211.3438
2,C10209,hSyn-AS-Gi-2,C,AS-mCherry,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000
3,C10208,hSyn-AS-Gi-2,A,AS-Gi,2,usb,205.9097,388.3886,356.8101,208.6715,203.0164,211.3438
4,C10210,hSyn-AS-Gi-3,B,AS-Gi,0,videofreeze,180.0000,4080.0000,4080.0000,180.0000,180.0000,180.0000


# Calibration & FreezeAnalysis

In [9]:
# going through experiment for this person
for experiment in experiment_cols[person]:
    print(experiment)
    # now go through each mouse for this person
    for index, mouse_row in df_start_info.iterrows():
        print(mouse_row["animal_id"])
        look_up_list = []
        look_up_list.append(experiment)
        
        for col in general_cols[person]:
            look_up_list.append(mouse_row[col])
        print(look_up_list)

        # find video given a list of strings it should contain
        vid_found = find_file_with_strings(PERSON_ROOT, look_up_list)
        print("vid_found: ", vid_found)
        # get experimental group
        experimental_group = mouse_row["experimental_group"]

        # Get fps of video
        vid_opencv_obj = cv2.VideoCapture(vid_found)
        fps = vid_opencv_obj.get(cv2.CAP_PROP_FPS)
        print("Frames per second:", fps)
        # if look_up_list contains the element 'usb', don't divide by 2,
        # otherwise, divide by 2, i think? yea bc eztrack downsamples by 2 for non-usbs
        if 'usb' in look_up_list:
            fps_eztrack_adjusted = fps
        else:
            fps_eztrack_adjusted = fps / 2 # bc eztrack downsamples by 2
        vid_opencv_obj.release()

        # get correction time in seconds to subtract from total frames
        correction_time_in_frames = math.ceil(time_to_frames(mouse_row[experiment], fps_eztrack_adjusted))
        print("correction_time_in_frames: ", correction_time_in_frames)

        # find calibration video
        chamber = mouse_row["chamber"]

        #to find calibration for extinction_get rid of _1 and _2
        if "_1" in experiment or "_2" in experiment:
            experiment_cali = experiment[:-2]
        else:
            experiment_cali = experiment

        calibration_vid_file_name = find_file_with_strings(CALIBRATION_ROOT, [chamber, experiment_cali])
        print("calibration_vid_file_name: ", calibration_vid_file_name)

        video_dict = {
            'dpath'   : CALIBRATION_ROOT,  
            'file'    : calibration_vid_file_name.split("/")[-1],
            'start'   : calibrate_video_what_frame_to_start, 
            'end'     : None,
            'dsmpl'   : dsmpl,
            'stretch' : dict(width=1, height=1),
            'cal_frms' : number_of_frames_to_calibrate
            }

        img_crp, video_dict = fz.LoadAndCrop(video_dict)
        ####### CALIBRATION #######
        cal_dif_avg, percentile, mt_cutoff = fz.calibrate_custom(video_dict, cal_pix=10000, SIGMA=1)

        ####### FREEZE ANALYSIS #######
        cap = cv2.VideoCapture(vid_found)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("length of vid: ", length)

        vid_d_end = length

        video_dict = {
            'dpath'   : PERSON_ROOT,  
            'file'    : vid_found.split("/")[-1],
            'fpath'   : vid_found,
            'start'   : vid_d_start, 
            'end'     : vid_d_end,
            'dsmpl'   : dsmpl,
            'stretch' : dict(width=1, height=1)
        }

        Motion, frames_processed = fz.Measure_Motion(video_dict, mt_cutoff, SIGMA=1)  
        plt_mt = hv.Curve((np.arange(len(Motion)),Motion),'Frame','Pixel Change').opts(
            height=h,width=w,line_width=1,color="steelblue",title="Motion Across Session")
        plt_mt

        Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  
        fz.SaveData(video_dict,Motion,Freezing,mt_cutoff,FreezeThresh,MinDuration)
        print('Average Freezing: {x}%'.format(x=np.average(Freezing)))

        vid_name_no_ext = vid_found.split("/")[-1].split(".")[0]
        freezing_result_filename = f"{vid_name_no_ext}_FreezingOutput.csv"
        freezing_result_path = vid_found.replace(vid_found.split("/")[-1], freezing_result_filename)

        # this processing includes frame correction from when experiment started
        processed_freezing_out_path = freezing_result_path.replace(".csv", "_processed.csv")

        df_freezing_out = pd.read_csv(freezing_result_path)
        #finding the timing file for this person and experiment
        timing_filepath = find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment])
        df_timing = ez.timing_file_processing(find_file_with_strings(PERSON_ROOT, [person.lower(), "FC_info", experiment]), fps_eztrack_adjusted, correction_time_in_frames)
        df_aligned = ez.freezing_alignment(df_freezing_out, df_timing)

        print("processed_freezing_out_path: ", processed_freezing_out_path)
        df_aligned.to_csv(processed_freezing_out_path, index=False)
        
        break
    break

conditioning
hSyn-AS-Gi-1
['conditioning', 'hSyn-AS-Gi-1', 'videofreeze']
['conditioning', 'hsyn-as-gi-1', 'videofreeze']
vid_found:  /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080822_conditioning/VideoFreeze/hSyn-AS-Gi-1.avi
Frames per second: 60.0
correction_time_in_frames:  5400
['a', 'conditioning']
calibration_vid_file_name:  /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/calibration/Chamber_C_calibration_conditioning.avi
file: /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/calibration/Chamber_C_calibration_conditioning.avi
total frames: 4740
nominal fps: 60.0
dimensions (h x w): 240,320
Average frame-by-frame pixel difference: 0.3381632307655912
99.99 percentile of pixel change differences: 4.314312587454654
Grayscale change cut-off for pixel change: 8.628625174909308
length of vid:  32394
cap_max: 32394
length of Motion: 32394
range with len(Motion) 32393


 50%|████▉     | 16196/32393 [00:18<00:18, 860.61it/s]


no. frames detected: 16196
no. frames not detected: 1
total frames processed: 16196

Average Freezing: 21.36947394418375%
['olena', 'fc_info', 'conditioning']
['olena', 'fc_info', 'conditioning']
   Trial    CS ON
0      1   5400.0
1      2   8100.0
2      3  11700.0
processed_freezing_out_path:  /media/rory/Padlock_DT/Fear_Conditioning_Control/Olena_Group/080822_conditioning/VideoFreeze/hSyn-AS-Gi-1_FreezingOutput_processed.csv
